# Generate the RepCode circuit

In [1]:
from result_saver import SaverProvider

provider = SaverProvider()

In [6]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout

DISTANCE = 3
ROUNDS = 10
DEVICE = "ibmq_mumbai"
LOGICAL = '0'
SHOTS = int(1e5)

code = RepetitionCodeCircuit(DISTANCE, ROUNDS)
backend = provider.get_backend(DEVICE)
layout = get_repcode_layout(DISTANCE, backend, _is_hex=False)

qc = code.circuit[LOGICAL]                                   

Finding the longest path starting from 27 qubits: 100%|██████████| 27/27 [00:00<00:00, 31757.21it/s]


# get_counts fake_provider

In [12]:
from qiskit.providers.fake_provider import FakeSherbrooke

counts = FakeSherbrooke().run(qc, shots=SHOTS).result().get_counts()
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_counts)

[('000 00 00 00 00 00 00 00 00 00 00', 55505), ('000 00 00 00 00 01 00 00 00 00 00', 2328), ('000 00 00 00 00 00 00 00 00 00 01', 2293), ('000 00 00 00 00 00 01 00 00 00 00', 2261), ('000 00 00 01 00 00 00 00 00 00 00', 2259), ('000 00 00 00 00 00 00 00 01 00 00', 2240), ('000 01 00 00 00 00 00 00 00 00 00', 2231), ('000 00 01 00 00 00 00 00 00 00 00', 2219), ('000 00 00 00 00 00 00 01 00 00 00', 2212), ('000 00 00 00 00 00 00 00 00 01 00', 2208), ('000 00 00 00 01 00 00 00 00 00 00', 2114), ('001 00 00 00 00 00 00 00 00 00 00', 1452), ('100 00 00 00 00 00 00 00 00 00 00', 1015), ('000 10 00 00 00 00 00 00 00 00 00', 834), ('000 00 00 00 00 00 00 00 10 00 00', 827), ('000 00 00 00 00 00 10 00 00 00 00', 810), ('000 00 00 00 00 10 00 00 00 00 00', 803), ('000 00 00 00 00 00 00 10 00 00 00', 797), ('000 00 00 00 00 00 00 00 00 00 10', 761), ('000 00 00 00 00 00 00 00 00 10 00', 755), ('010 00 00 00 00 00 00 00 00 00 00', 754), ('000 00 00 00 10 00 00 00 00 00 00', 725), ('000 00 10 00 00

# AER simulator get_counts

In [13]:
from qiskit_aer import AerSimulator

counts = AerSimulator().run(qc, shots=SHOTS).result().get_counts()

sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_counts)

[('000 00 00 00 00 00 00 00 00 00 00', 100000)]


In [24]:
from qiskit.providers.aer.noise import NoiseModel, pauli_error

def get_general_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE, num_qubits):
    # Single-qubit errors
    reset_error = pauli_error([('X', pRE), ('I', 1 - pRE)])
    measure_error = pauli_error([('X', pRO), ('I', 1 - pRO)])
    u3_error = pauli_error([('X', p1Q / 3), ('Y', p1Q / 3), ('Z', p1Q / 3), ('I', 1 - p1Q)])
    idle_error = pauli_error([('X', pXY / 2), ('Y', pXY / 2), ('I', 1 - pXY)])
    idle2_error = pauli_error([('Z', pZ), ('I', 1 - pZ)])

    # Two-qubit error for 'cx' and 'swap'
    cx_error = pauli_error([('II', 1 - p2Q)] + [(i + j, p2Q / 15) for i in 'IXYZ' for j in 'IXYZ' if i + j != 'II'])
    swap_error = pauli_error([('II', 1 - p2Q)] + [(i + j, p2Q / 15) for i in 'IXYZ' for j in 'IXYZ' if i + j != 'II'])

    # Create a noise model
    noise_model = NoiseModel()
    for qubit in range(num_qubits):
        # Apply single-qubit errors to each qubit
        noise_model.add_quantum_error(reset_error, 'reset', [qubit])
        noise_model.add_quantum_error(measure_error, 'measure', [qubit])
        noise_model.add_quantum_error(u3_error, ['u3', 'u2'], [qubit])
        noise_model.add_quantum_error(idle_error, 'idle_1', [qubit])
        noise_model.add_quantum_error(idle2_error, 'idle_2', [qubit])

    # Apply two-qubit errors to all pairs of qubits
    for qubit1 in range(num_qubits):
        for qubit2 in range(num_qubits):
            if qubit1 != qubit2:
                noise_model.add_quantum_error(cx_error, 'cx', [qubit1, qubit2])
                noise_model.add_quantum_error(swap_error, 'swap', [qubit1, qubit2])

    return noise_model


In [25]:
noise_model = get_general_noise_model(p1Q=1e-4, p2Q=6e-3, pXY=1e-4, pZ=1e-4, pRO=1e-1, pRE=0, num_qubits=qc.num_qubits)
counts = AerSimulator(noise_model = noise_model).run(qc, shots=SHOTS).result().get_counts()

sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_counts)

[('000 00 00 00 00 00 00 00 00 00 00', 7394), ('000 10 00 00 00 00 00 00 00 00 00', 930), ('000 01 01 01 01 01 01 01 01 00 00', 905), ('000 10 10 10 10 10 10 10 10 10 00', 896), ('000 01 01 01 01 01 01 01 01 01 00', 890), ('000 01 01 01 01 01 01 01 01 01 01', 870), ('000 01 01 01 01 00 00 00 00 00 00', 868), ('000 01 01 00 00 00 00 00 00 00 00', 865), ('000 10 10 10 10 00 00 00 00 00 00', 857), ('000 10 10 10 10 10 00 00 00 00 00', 854), ('001 00 00 00 00 00 00 00 00 00 00', 853), ('000 01 01 01 01 01 00 00 00 00 00', 850), ('000 01 01 01 01 01 01 01 00 00 00', 841), ('000 01 01 01 00 00 00 00 00 00 00', 840), ('000 01 01 01 01 01 01 00 00 00 00', 838), ('000 10 10 10 00 00 00 00 00 00 00', 835), ('000 10 10 10 10 10 10 00 00 00 00', 834), ('010 00 00 00 00 00 00 00 00 00 00', 830), ('000 10 10 00 00 00 00 00 00 00 00', 824), ('000 10 10 10 10 10 10 10 10 00 00', 820), ('000 01 00 00 00 00 00 00 00 00 00', 818), ('000 10 10 10 10 10 10 10 10 10 10', 817), ('100 00 00 00 00 00 00 00 00 